# The HFM library - A fast marching solver with adaptive stencils

## Part : Applications
## Chapter : Computation and usage of the medial axis

In [2]:
",".join(("a","b"))

'a,b'

In [5]:
",".join(str(e) for e in (2,3))

'2,3'

In [6]:
l=[1,2]
l.append(3)

In [7]:
l

[1, 2, 3]

In [10]:
{1:1}.get(2,3)

3